In [1]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [10]:
import netCDF4
import matplotlib.pyplot as plt
import datetime 
import statsmodels as statsmodels
import statistics as stats
import decimal
import os
import numpy as np

In [96]:


class ReadData:
    """
    ReadData-- generates the dataset
    
    TODO: currently doesn't use generators, so it can be slow.
    """
    DMI = {}
    sortedKeys_DMI = []
    TRMM_data = []
    ElNino_data= []
    sortedKeys_Nino = [[],[],[],[]] #currently, we have 4 Nino items
    data_dict = {}
    
    #constants
    begin = datetime.date(1998, 1, 1) #begin of dates we look at
    end = datetime.date(2019, 12, 30) #end of dates we look at
    
    
    
    
    def __init__(self):
        
        #in the init phase, we generated each data set and sort the dictionary keys
        
        self.DMI_data = self.generateDataDMI()
        print("DMI data generated")
        self.findMinData(self.DMI_data)
        self.sortedKeys_DMI = self.sortKeys(self.DMI_data)   
        
        self.TRMM_data = self.generateDataTrim()
        self.sortedKeys_DMI = self.sortKeys(self.TRMM_data)   
        print("TRMM data generated")
        
        self.ElNino_data = self.generateDataElNino()
        for i in range(0,len(self.ElNino_data)):
            self.sortedKeys_Nino = self.sortKeys(self.ElNino_data[i])
        print("ELNino data generated")
        
        
        
    def sortKeys(self,data):
        return sorted(data,key = lambda key: self.toDateTime(key )-self.begin)
    def findMinData(self,data):
        v = 0
        for key in data.keys():
            diff = self.toDateTime(key )-self.begin
            if(v == 0 or v> diff):
                v = diff
        print(v)
        return v
        
        
    
    """
    Generate DMI dataset--used for (???)
    """
    def generateDataDMI(self):
        iodfile = "dmi.nc" #were reading the DMI file
        f = netCDF4.Dataset(iodfile, 'r')
        SSTAnp=f.variables['DMI'][:]
        weeknp=f.variables['WEDCEN2'][:]
        SSTA=SSTAnp.tolist() #list of DMI SSTA
        week=weeknp.tolist() 
        f.close()
        DMIdate=[]     #DATE OF DMI 
        for i in range(0, len(week)):    
            days=week[i]
            start=datetime.date(1900, 1, 1)
            delta=datetime.timedelta(days)
            offset=start+delta
            DMIdate.append(offset)

        date_DMI={}
        for j in range(0, len(SSTA)):
            if(self.isInTimeRange(DMIdate[j])):
                date_DMI[str(DMIdate[j]).replace("-","")] =  SSTA[j]
            else:
                pass
                #this data we will just discard for now
        return date_DMI
    
    def generateDataElNino(self):
        ninoindicesfile="elnino indices.txt"
        f=open(ninoindicesfile, "r")
        linelist=list(f.readlines())    
        #### ARRAYS OF FORMAT (DATETIME, DMI)
        date_Ninos=[{},{},{},{}]
        #12,3,34,4 are the ninos we take
        
        search_range = [[19,23],[32,36],[45,49],[58,62]]
        for j in range(0,len(search_range)):
            for i in range(4, len(linelist)):
                date_str = self.replaceDateTime(linelist[i][1:10])
                if (self.isInTimeRangeStr(date_str)):
                    date_Ninos[j][date_str] =  decimal.Decimal(linelist[i][search_range[j][0]:search_range[j][1]])            
                else:
                    pass
                    #this is the date we discard
        f.close() 

        
        #Not sure what to return here, potentially a dictionary would be more useful
        return date_Ninos
    """
    Generate TRMM dataset
    """
    def generateDataTrim(self): #this is very slow, increase speed
        TRMMfolder="TRMM1/"
        datetime_prec={} #create a dictionary with the dates
        for filename in os.listdir(TRMMfolder):
            if filename.endswith('nc4'):
                f=netCDF4.Dataset(TRMMfolder + str(filename))
                precp=f.variables['precipitation'][:]
                date_str = str(filename[11:19])
                if(self.isInTimeRangeStr(date_str)):
                    datetime_prec[date_str] =  precp
                else:
                    print(date_str)
                f.close()
            else:
                pass
        
        #datetime_prec=np.array(datetime_prec)
        return datetime_prec
    def isInTimeRange(self,time):
        if (time >= self.begin and time <= self.end):
            return True
        return False
    def toDateTime(self,time):
        return datetime.date(int(time[0:4]),int(time[4:6]),int(time[6:8]))
    def isInTimeRangeStr(self,time):
        return self.isInTimeRange(self.toDateTime(time))
        
        
    def replaceDateTime(self,string):
        return str(datetime.datetime.strptime(string, '%d%b%Y')).split(" ")[0].replace("-","")
    def generateDataSNHT(self,dataset):   #SNHT FOR time series of dimension 1 x n
        n=len(dataset)
        sd=stats.pstdev(dataset)
        snhtset=[]
        for y in range(1,n-1): #Loops from Week 1 to week n-1
            summ1=0
            for i in range(1, y+1):
                summ1 += decimal.Decimal((dataset[i] - decimal.Decimal(stats.mean(dataset)))/sd)
            z_1= decimal.Decimal(1/(y+1)) * summ1
            summ2=0
            for i in range(y+1, n):
                summ2 += decimal.Decimal((dataset[i] - decimal.Decimal(stats.mean(dataset)))/sd)
            z_2=decimal.Decimal(1/(n-y)) *summ2
            snhtset.append(y * (z_1 **2) + (n-y)*(z_2**2))
        return snhtset
    
        
    
    #TODO: stort this data into three different arrays given by a dictionary entry
    





reader = ReadData()
    









DMI data generated
6 days, 0:00:00
TRMM data generated
ELNino data generated


In [54]:
reader.TRMM_data.keys()
#note, days are actually not matched, so we need to find the offset
#and then combine the one-to-one parts of the equations

dict_keys(['20090313', '20071202', '20110528', '20140411', '20100424', '20150806', '20150201', '20080818', '20031001', '20090125', '20080503', '20100612', '20090914', '20140627', '20180508', '20000619', '20040306', '19981103', '20121106', '20180813', '20010715', '20050720', '20190404', '20000105', '20040130', '20180214', '20040901', '20190318', '20050516', '20010523', '20171209', '20160303', '20001103', '19980619', '20160629', '20181212', '20130710', '19990715', '20170725', '20181024', '20170808', '19980105', '20170513', '19990523', '20130526', '20191128', '20160904', '20070518', '20151207', '20091123', '20070803', '20060414', '20020421', '20070204', '20151031', '20111004', '20020617', '20060308', '20060622', '20101108', '20081219', '20080609', '20080323', '20100518', '20021207', '20110414', '20150421', '20090705', '20100803', '20100204', '20021031', '20090828', '20061004', '20080115', '20150617', '20110308', '20110622', '20080924', '20071108', '20010303', '20171103', '20010629', '2004

In [55]:
reader.DMI_data.keys()

dict_keys(['19811104', '19811111', '19811118', '19811125', '19811202', '19811209', '19811216', '19811223', '19811230', '19820106', '19820113', '19820120', '19820127', '19820203', '19820210', '19820217', '19820224', '19820303', '19820310', '19820317', '19820324', '19820331', '19820407', '19820414', '19820421', '19820428', '19820505', '19820512', '19820519', '19820526', '19820602', '19820609', '19820616', '19820623', '19820630', '19820707', '19820714', '19820721', '19820728', '19820804', '19820811', '19820818', '19820825', '19820901', '19820908', '19820915', '19820922', '19820929', '19821006', '19821013', '19821020', '19821027', '19821103', '19821110', '19821117', '19821124', '19821201', '19821208', '19821215', '19821222', '19821229', '19830105', '19830112', '19830119', '19830126', '19830202', '19830209', '19830216', '19830223', '19830302', '19830309', '19830316', '19830323', '19830330', '19830406', '19830413', '19830420', '19830427', '19830504', '19830511', '19830518', '19830525', '1983